In [3]:
%pylab inline
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [6]:
transactions = pd.read_csv('train.csv')
macro = pd.read_csv('macro.csv')
test = pd.read_csv('test.csv')

In [13]:
train = pd.merge(transactions, macro, on='timestamp', how='left')

In [16]:
test_all = pd.merge(test, macro, on='timestamp', how='left')

In [26]:
test_all.shape, train.shape

((7662, 390), (30471, 391))

## Feature selection

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [62]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

def classification_metrics(y_test, y_pred):
    funcs = dict(
        accuracy_score=accuracy_score, 
        precision_score=precision_score, 
        recall_score=recall_score,
        f1_score=f1_score,
        roc_auc_score=roc_auc_score,
    )
    result = {k: v(y_test, y_pred) for k, v in funcs.items()}
    for k, v in result.items():
        print(k.title(), v)
    return result

In [63]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, median_absolute_error, r2_score

def regression_metrics(y_test, y_pred):
    funcs = dict(
        explained_variance_score=explained_variance_score, 
        mean_absolute_error=mean_absolute_error, 
        mean_squared_error=mean_squared_error,
        median_absolute_error=median_absolute_error,
        r2_score=r2_score,
    )
    result = {k: v(y_test, y_pred) for k, v in funcs.items()}
    for k, v in result.items():
        print(k.title(), v)
    return result

In [75]:
numerical_only = train.select_dtypes(exclude=['object'])
numerical_only.replace(to_replace=NaN, value=-1, inplace=True)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [76]:
X, y = numerical_only[list(set(numerical_only.columns) - {'price_doc'})], numerical_only['price_doc']

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25, random_state=42)
clf = RandomForestRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [88]:
m = regression_metrics(y_test, y_pred)

Explained_Variance_Score 0.604661806257
Mean_Absolute_Error 1582215.11461
Mean_Squared_Error 9.3153513783e+12
Median_Absolute_Error 747000.0
R2_Score 0.604111322614


In [84]:
ranking = sorted(zip(X.columns, clf.feature_importances_), key=lambda x: -x[1])

In [89]:
for f, v in ranking[:30]:
    print(f, v)

full_sq 0.395262016644
cafe_count_3000 0.0490838723961
nuclear_reactor_km 0.0187570344698
office_sqm_5000 0.0183306367857
cafe_count_3000_price_2500 0.0181813554288
cafe_count_5000_price_2500 0.0157776434302
sport_count_3000 0.0146928084855
theater_km 0.0138408473061
radiation_km 0.0130450630135
cafe_count_5000_price_high 0.0115623379061
hospital_beds_raion 0.0097537453267
church_count_5000 0.0093421863211
cafe_count_5000_price_4000 0.00826254565741
stadium_km 0.00768139468416
ttk_km 0.00767870873112
micex_cbi_tr 0.00758966149587
cafe_count_2000 0.00659786292806
rts 0.00592712968395
kindergarten_km 0.00519559872362
micex_rgbi_tr 0.00513827195788
floor 0.00512338831071
swim_pool_km 0.00509232207152
big_church_km 0.00504610726838
railroad_station_avto_km 0.00466974692631
public_healthcare_km 0.00455336307724
brent 0.00455257018678
zd_vokzaly_avto_km 0.00446350074313
usdrub 0.00442157252668
cafe_count_1000 0.00431675609973
state 0.00415452791582


Total area in square meters is being the most important feature with huge gap (not surprised)